In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32950 non-null  int64  
 1   job             32950 non-null  object 
 2   marital         32950 non-null  object 
 3   education       32950 non-null  object 
 4   default         32950 non-null  object 
 5   housing         32950 non-null  object 
 6   loan            32950 non-null  object 
 7   contact         32950 non-null  object 
 8   month           32950 non-null  object 
 9   day_of_week     32950 non-null  object 
 10  duration        32950 non-null  int64  
 11  campaign        32950 non-null  int64  
 12  pdays           32950 non-null  int64  
 13  previous        32950 non-null  int64  
 14  poutcome        32950 non-null  object 
 15  emp.var.rate    32950 non-null  float64
 16  cons.price.idx  32950 non-null  float64
 17  cons.conf.idx   32950 non-null 

In [4]:
test = pd.read_csv('test.csv')

In [5]:
target = train.y
test_target = test.y
inputs = train.drop('y',axis='columns')
test_inputs = test.drop('y',axis = 'columns')

checking for null values

In [6]:
target.isnull().values.any()

False

In [7]:
new_target = target.map({'yes':1,'no':0})
Test_target = test_target.map({'yes':1,'no':0})

In [8]:
inputs.job.unique()
inputs.education.unique()

array(['high.school', 'university.degree', 'basic.9y',
       'professional.course', 'basic.4y', 'basic.6y', 'unknown',
       'illiterate'], dtype=object)

checking whether there are any **NaN** values

In [9]:
inputs.columns[inputs.isna().any()];
test_inputs.columns[test_inputs.isna().any()]

Index([], dtype='object')

In [10]:
inputs.columns == test_inputs.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
scorer = {}

In [13]:
for i in inputs.columns:
    if inputs[i].dtypes == 'object':
        temp_var = i+"_le"
        temp_labelencoder = LabelEncoder()
        inputs[temp_var] = temp_labelencoder.fit_transform(inputs[i])
        scorer[i] = list(temp_labelencoder.classes_)
        test_inputs[temp_var] = temp_labelencoder.fit_transform(test_inputs[i])

In [15]:
scorer;

In [16]:
import json
with open("scorer.json",'w') as fp:
    json.dump(scorer, fp)

In [17]:
new_df = inputs.drop([i for i in inputs.columns if inputs[i].dtypes == 'object'], axis='columns')
test_df = test_inputs.drop([i for i in test_inputs.columns if test_inputs[i].dtypes == 'object'], axis='columns')
new_df.head(3)
test_df.head(3)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_le,marital_le,education_le,default_le,housing_le,loan_le,contact_le,month_le,day_of_week_le,poutcome_le
0,61,268,3,999,0,-1.8,92.893,-46.2,1.354,5099.1,9,0,5,0,2,0,0,6,1,1
1,58,75,7,999,0,1.4,93.444,-36.1,4.964,5228.1,9,0,5,0,0,0,0,1,0,1
2,30,166,1,999,0,1.4,93.918,-42.7,4.962,5228.1,0,1,3,0,2,0,0,3,4,1


### Training the model

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(new_df,new_target, test_size = 0.2)

In [20]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()

In [21]:
model.fit(X_train,Y_train)

BernoulliNB()

In [150]:
model.score(X_test,Y_test)

0.8389984825493172

In [167]:
model.predict(test_df[0:1])

array([0])

### score with the test.csv

In [174]:
model.score(test_df,Test_target)

0.8319980577810148

### Saving the model

In [19]:
import pickle
# pickle.dump(model,open('model','wb'))